# SQL Templates for Interview Prep
Below are SQL templates using window functions like `ROW_NUMBER()`, `RANK()`, `DENSE_RANK()`, `NTILE()`, `SUM() OVER`, and `LAG()/LEAD()`. 

## 1. ROW_NUMBER() for Ranking Within Groups
**Purpose**: Assigns a unique rank to each row within a partition, useful for identifying top-N records per group (e.g., top products by region).

**Template**:
```sql
WITH RankedData AS (
    SELECT column1, column2, AGGREGATE_FUNCTION(metric) as metric_value,
           ROW_NUMBER() OVER (PARTITION BY partition_column ORDER BY metric DESC) as rn
    FROM table_name
    GROUP BY column1, column2, partition_column
)
SELECT column1, column2, metric_value
FROM RankedData
WHERE rn <= N;
```

**Example** (Top 2 employees by sales per department):
```sql
WITH RankedSales AS (
    SELECT e.name, e.department, SUM(s.sales_amount) as total_sales,
           ROW_NUMBER() OVER (PARTITION BY e.department ORDER BY SUM(s.sales_amount) DESC) as rn
    FROM employees e
    JOIN sales s ON e.id = s.employee_id
    GROUP BY e.id, e.name, e.department
)
SELECT name, department, total_sales
FROM RankedSales
WHERE rn <= 2;
```
**When to Use**: When you need unique rankings (no ties) within groups, like top performers or products.

## 2. RANK() for Ranking with Ties
**Purpose**: Assigns ranks to rows, with tied values getting the same rank and leaving gaps in the sequence (e.g., 1, 1, 3).

**Template**:
```sql
SELECT column1, column2, metric,
       RANK() OVER (PARTITION BY partition_column ORDER BY metric DESC) as rank
FROM table_name
WHERE rank <= N;
```

**Example** (Rank products by revenue, allowing ties):
```sql
SELECT p.product_name, s.region, SUM(s.revenue) as total_revenue,
       RANK() OVER (PARTITION BY s.region ORDER BY SUM(s.revenue) DESC) as rank
FROM sales s
JOIN products p ON s.product_id = p.id
GROUP BY p.product_name, s.region
HAVING RANK() OVER (PARTITION BY s.region ORDER BY SUM(s.revenue) DESC) <= 3;
```
**When to Use**: When ties are expected, and you want gaps in rank (e.g., two products tied at rank 1, next is rank 3).

## 3. DENSE_RANK() for Ranking Without Gaps
**Purpose**: Similar to `RANK()`, but ranks are consecutive (e.g., 1, 1, 2) for ties.

**Template**:
```sql
SELECT column1, column2, metric,
       DENSE_RANK() OVER (PARTITION BY partition_column ORDER BY metric DESC) as dense_rank
FROM table_name
WHERE dense_rank <= N;
```

**Example** (Top 3 stores by sales volume, no rank gaps):
```sql
SELECT store_id, region, SUM(sales_volume) as total_volume,
       DENSE_RANK() OVER (PARTITION BY region ORDER BY SUM(sales_volume) DESC) as dense_rank
FROM store_sales
GROUP BY store_id, region
HAVING DENSE_RANK() OVER (PARTITION BY region ORDER BY SUM(sales_volume) DESC) <= 3;
```
**When to Use**: When you need consecutive ranks for ties, common in reports where gaps look awkward.

## 4. Running Total with SUM() OVER
**Purpose**: Calculates cumulative totals over a sequence, often for time-series or ordered data.

**Template**:
```sql
SELECT column1, date_column, metric,
       SUM(metric) OVER (PARTITION BY partition_column ORDER BY order_column ROWS UNBOUNDED PRECEDING) as running_total
FROM table_name
ORDER BY partition_column, order_column;
```

**Example** (Running total of expenses by department over time):
```sql
SELECT department, expense_date, amount,
       SUM(amount) OVER (PARTITION BY department ORDER BY expense_date ROWS UNBOUNDED PRECEDING) as running_total
FROM expenses
ORDER BY department, expense_date;
```
**When to Use**: For cumulative metrics, like tracking budget spend or sales over time.

## 5. NTILE() for Dividing Data into Buckets
**Purpose**: Divides rows into N equal buckets (e.g., quartiles, deciles) within a partition, useful for segmentation.

**Template**:
```sql
SELECT column1, column2, metric,
       NTILE(N) OVER (PARTITION BY partition_column ORDER BY metric) as bucket
FROM table_name;
```

**Example** (Segment customers into 4 quartiles by purchase amount):
```sql
SELECT c.customer_name, c.region, SUM(o.purchase_amount) as total_purchases,
       NTILE(4) OVER (PARTITION BY c.region ORDER BY SUM(o.purchase_amount) DESC) as quartile
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.customer_name, c.region;
```
**When to Use**: For customer segmentation or performance banding (e.g., top 25% of sales reps).

## 6. LAG()/LEAD() for Comparing Rows
**Purpose**: Accesses previous or next row’s data within a partition, great for time-series comparisons.

**Template**:
```sql
SELECT column1, date_column, metric,
       LAG(metric) OVER (PARTITION BY partition_column ORDER BY order_column) as previous_value,
       LEAD(metric) OVER (PARTITION BY partition_column ORDER BY order_column) as next_value
FROM table_name;
```

**Example** (Compare monthly sales to previous month):
```sql
SELECT region, month, SUM(sales_amount) as sales,
       LAG(SUM(sales_amount)) OVER (PARTITION BY region ORDER BY month) as prev_month_sales
FROM sales
GROUP BY region, month
ORDER BY region, month;
```
**When to Use**: For month-over-month or sequential comparisons.

## Prep Tips
- **Practice**: Use LeetCode Database (e.g., 'Department Top Three Salaries') or StrataScratch for business-like problems. Aim for 10-15 window function problems.
- **Edge Cases**: Handle NULLs, empty partitions, or ties explicitly (e.g., use `COALESCE` or check for zero rows).
- **Explain**: During interviews, clarify the partition and order logic (e.g., 'I’m partitioning by region to group sales, ordering by amount for ranking').
- **Tools**: Practice in DB-Fiddle or SQLZoo to simulate LeetCode’s SQL environment.